# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [2]:
try:
    data = pd.read_csv('insurance.csv')    
except:
    print('Что-то пошло не так...')

In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


В выборке 5000 объектов. Пропусков в данных нет. Тип данных представлен корректно.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?**

**Ответ:** Не изменится.

**Обоснование:** \
\
Пусть $P$ обратимая матрица, на которую умнажается $X$. Тогда формула для вычисления вектора весов линейной регрессии будет иметь вид:

$$
w = ((XP)^T (XP))^{-1} (XP)^T y = (P^T X^T (XP))^{-1} (XP)^T y = (P^T (X^T XP))^{-1} (XP)^T y = (X^T X P)^{-1} (P^T)^{-1} P^T X^T y = (X^T X P)^{-1} X^T y = P^{-1} (X^T X)^{-1} X^T y
$$

Предсказания $a_1$ для $XP$ вычисляются по формуле:

$$
a_1 = XPw = XP(P^{-1} (X^T X)^{-1} X^T y) = X(PP^{-1})(X^T X)^{-1} X^T y = X E (X^T X)^{-1} X^T y = Xw = a
$$


Т.к. $a = a_1$ умножение признаков на обратимую матрицу не меняет качество линейной регрессии.

## Алгоритмы преобразования 

Предложите алгоритм преобразования данных для решения задачи.

Умножение матрицы признаков на обратимую матрицу является очень хорошим вариантом решения задачи. Но попробуем исследовать и другие алгоритмы.

### Алгоритм

**Признаки умножаются на действительное число**

**Обоснование**

Пусть $k$ действительное число, тогда:

$$
w = ((kX)^T(kX))^{-1} (kX)^T y = (kX^T kX)^{-1} kX^T y = (k^2 X^T X)^{-1} kX^T y = k^{-2}(X^T X)^{-1}kX^T y = k^{-1}(X^TX)^{-1}X^T y
$$

Предсказания $a_1$ для $kX$ вычисляются по формуле:

$$
a_1 = kXk^{-1}(X^TX)^{-1}X^Ty = X(X^TX)^{-1}X^Ty = a
$$

Т.к. $a = a_1$ умножение признаков на действительное число $k$ не меняет качество линейной регрессии.

### Проверка алгоритма 

Делим данные на обучающую и валидационную выборки

In [5]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)

Проверяем качество модели до преобразования

In [6]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
print(r2_score(target_valid, predictions))

0.43522757127025646


Умножаем признаки обучающей и валидационной выборки на случайное число в диапозоне [0, 1)

In [7]:
k = np.random.rand()
features_train_k = features_train.values * k
features_valid_k = features_valid.values * k

In [8]:
model.fit(features_train_k, target_train)
predictions = model.predict(features_valid_k)
print(r2_score(target_valid, predictions))

0.4352275712702719


r2 метрика не изменилась

### Алгоритм

**К признакам применяется векторная функция**

$$
f(X) = k(value) + b
$$

Где $k$ и $b$ любые действительные числа.

**Обоснование**\
Вектор предсказаний $a$ представляет собой произведение матрицы признаков $X$ на вектор коэффициентов $w$. 

$$
a=(w_0 + x_{11}w_1 + ... + x_{1n}w_n, ... , w_0 + x_{m1}w_1 + ... + x_{mn}w_n)
$$

Т.е. каждый элемент вектора $a$ представляет собой линейную функцию $n$ переменных $x = (x_1, x_2, x_3, ... , x_n)$, где $x$ это элементы матрицы признаков $X$.\
Метод предсказаний модели основан на свойстве линейных функций: приращение функции пропорционально приращению аргумента. \
Т.о. если удастся доказать, что после применение нашей векторной функции к матрице признаков, элементы вектора $a$ сохранят вид линейных функций, то и качество линейной регрессии не изменится.

$a$ примет вид

$$
a=(w_0 + (kx_{11} + b)w_1 + ... + (kx_{1n} + b)w_n, ... , w_0 + (kx_{m1} + b)w_1 + ... + (kx_{mn} + b)w_n)
$$

$$
a=(w_0 + kw_1x_{11} + bw_1 + ... + kw_nx_{1n} + bw_n, ... , w_0 + kw_1x_{m1} + bw_1 + ... + kw_nx_{mn} + bw_n)
$$

$$
a=((w_0 + bw_1 + ... + bw_n)+ kw_1x_{11} +...+ kw_nx_{1n}, ... , (w_0 + bw_1 + ... + bw_n) + kw_1x_{m1} ... + kw_nx_{mn})
$$

\
\
Т.к. $w_i, b, k \in \mathbb{R}$ , пусть $ (w_0 + bw_1 + ... + bw_n)=c_0,  kw_1 = c_1, kw_2 = c_2, ... , kw_n = c_n, c_i \in \mathbb{R}$.\
\
Тогда

$$
a_i = c_0 + x_{i1}c_1 + x_{i2}c_2 + ... + x_{in}c_n
$$

После преобразования все элементы $a$ сохранили вид линейных функций.

### Проверка алгоритма 

In [9]:
def transform(values, k, b):   
    return k*values+b

In [10]:
k=np.random.rand()
b=np.random.rand()

In [11]:
features_train_f = transform(features_train.values, k, b)
features_valid_f = transform(features_valid.values, k, b)

In [12]:
model.fit(features_train_f, target_train)
predictions = model.predict(features_valid_f)
print(r2_score(target_valid, predictions))

0.43522757127027256


Качество линейной регрессии не изменилось.

### Алгоритм

**Умножение матрицы признаков на обратимую матрицу**

**Обоснование**

Как было доказано выше, данное преобразование не изменит качество линейной регрессии

### Проверка алгоритма 

Создадим случайную матрицу и проверим ее на обратимость

In [13]:
k=np.random.normal(size=(4, 4))

In [14]:
np.linalg.inv(k)

array([[ 0.16241751,  0.25517971,  0.04425054,  0.42196078],
       [-0.13394521,  0.42407156, -0.15437058,  0.07430148],
       [ 0.10871812,  0.50076054,  0.58965904, -0.06871399],
       [ 0.48154665,  0.35351769, -0.36087847,  0.1235782 ]])

Умножим матрицы признаков обучающей и валидационной выборок на обратимую марицу и проверим качество модели

In [15]:
features_train_mult = features_train.values @ k
features_valid_mult = features_valid.values @ k

In [16]:
model.fit(features_train_mult, target_train)
predictions = model.predict(features_valid_mult)
print(r2_score(target_valid, predictions))

0.4352275712705401


Качество линейной регрессии не изменилось

## Вывод

Было исследовано три алгоритма преобразования:

- Признаки умножаются на действительное число
- К признакам применяется векторная функция $f(X) = k(value) + b$ , где $k$ и $b$ любые действительные числа.
- Умножение матрицы признаков на обратимую матрицу

Ни один из перечисленных алгоритмов преобразования данных не меняет качества модели *LinearRegression*.\
Наибольшую защиту персональных данных дает умножение матрицы признаков на обратимую матрицу.